## 論文実装
https://publicacoes.softaliza.com.br/cilamce/article/view/8202

A Multi-Objective Ant Colony Optimization for Routing in Printed Circuit Boards


In [ ]:
import sys
import math
from heapq import *
from collections import defaultdict
import random
import networkx as nx
import matplotlib.pyplot as plt
from typing import List
sys.setrecursionlimit(10**7)

from graph import Graph_2d, Graph_3d
from algo import Algo
from visualization import *

---
### Utils

In [ ]:
def dijkstra(start, n, to):
    prevs = [0 for _ in range(n*n)]
    d = [float('inf') for _ in range(n*n)]
    d[start] = 0
    q = [(0, start)]
    while q:
        (ci, i) = heappop(q)
        if d[i] < ci:
            continue
        for j in to[i]:
            if d[j] > d[i] + 1:
                d[j] = d[i] + 1
                prevs[j] = i
                heappush(q, (d[j], j))
    return d, prevs

def get_path(s, v, prevs):
    '''
    s -> vへの最短経路path
    '''
    if v == s:
        return [s]
    path = [v]
    while True:
        path.append(prevs[v])
        v = prevs[v]
        if v == s:
            return path


In [ ]:
def nodenum_to_coord(grid_size, nodenum):
    z = nodenum // (grid_size * grid_size)
    rem = nodenum % (grid_size * grid_size)
    y = rem // grid_size
    x = rem % grid_size
    return x, y, z


def counting_corner(grid_size ,path):
    """
    pathの曲がり回数をカウントする
    """
    corner_count = 0
    path_coordinates = []
    for p in path:
        x_, y_, z_ = nodenum_to_coord(grid_size, p)
        path_coordinates.append((x_, y_, z_))

    for i in range(len(path_coordinates)):
        if i == 0 or i == len(path_coordinates)-1:
            # nop
            pass
        else:
            vec_1_x = path_coordinates[i][0] - path_coordinates[i-1][0]
            vec_1_y = path_coordinates[i][1] - path_coordinates[i-1][1]
            vec_1_z = path_coordinates[i][2] - path_coordinates[i-1][2]
            vec_1 = (vec_1_x, vec_1_y, vec_1_z)

            vec_2_x = path_coordinates[i+1][0] - path_coordinates[i][0]
            vec_2_y = path_coordinates[i+1][1] - path_coordinates[i][1]
            vec_2_z = path_coordinates[i+1][2] - path_coordinates[i][2]
            vec_2 = (vec_2_x, vec_2_y, vec_2_z)

            if vec_1 != vec_2:
                corner_count += 1

    return corner_count

---
### 3次元検証

目的関数をいろいろ改変したものに名前を付ける。

- original
  - 原論文のアルゴリズム。目的関数は以下の3つ
    1. 経路長
    2. 交差点数
    3. 長さ整合
- add_corner_constraint
  - 原論文の目的関数に、さらに「曲げ回数」の最小化も入れたもの。
    1. 経路長
    2. 交差点数
    3. 長さ整合 
    4. 曲げ回数

In [ ]:
algo_name = "original"

In [ ]:
grid_size = 10
graph_3d = Graph_3d(grid_size=grid_size)
algo = Algo(grid_size=grid_size, dim=3, to=graph_3d.to, algo_name=algo_name)

In [ ]:
def main_3d_inspection(pairs, param_name, w1, w2, w3, w4=None, do_visualize=False):
    """
    Args:
        pairs: スタート地点・ゴール地点の組
        param_name: 重みパラメータの名前
        w1, w2, w3: param_name
    """
    # --- 実行部：ペア指定と可視化 ---
    sd = 0
    random.seed(sd)
    # random.seed(0)
    # random.seed(1)
    # random.seed(2)

    # paths = algo.equal_length_routing(pairs, w1=w1, w2=w2, w3=w3)
    paths = algo.equal_length_routing(pairs, w1=w1, w2=w2, w3=w3, w4=w4)

    print("======================================================================================")
    print(f"パラメータ名：{param_name}")
    print(f"seed値：{sd}")

    # 結果の表示(1本ずつの表示)
    for i, path in enumerate(paths):
        print(f"w1: {w1}, w2: {w2}, w3: {w3}")
        print('len(path) =', len(path))
        if i == 0:
            path1_length = len(path)
        elif i == 1:
            path2_length = len(path)
        elif i == 2:
            path3_length = len(path)


        savefilename = f"result_jikken_1_{param_name}_w1_{w1}_w2_{w2}_w3_{w3}_length_{len(path)}_pathNo_{i}.html"
        if do_visualize:
            vis_gridpath_3d(grid_size, [path], obs_nodes=graph_3d.obs, saveflg=False, savefilename=savefilename)

    # 結果の表示(3本まとめて表示)
    savefilename = f"result_jikken_1_{param_name}_w1_{w1}_w2_{w2}_w3_{w3}_pathall.html"
    if do_visualize:
        vis_gridpath_3d(grid_size, paths, obs_nodes=graph_3d.obs, saveflg=False, savefilename=savefilename)


    # 交差点数を勘定    
    crossdict = defaultdict(int)
    for path in paths:
        for p in path:
            crossdict[p] += 1
    n_cross = sum(crossdict.values()) - len(crossdict.keys())
    print(f'組み合わせ名：{param_name}, 交差数: {n_cross}' )


    # すべてのpathの曲げ回数の和を計算
    n_corner = 0
    for path in paths:
        n_corner += counting_corner(grid_size, path)

    return param_name, sd, path1_length, path2_length, path3_length, n_cross, paths, n_corner


    

In [ ]:
# # 重み設定　（論文の目的関数）
# weight_sets = {
#     "combination_1": {"w1": 10, "w2": 45, "w3": 45},
#     "combination_2": {"w1": 10, "w2": 5, "w3": 3},
#     "combination_3": {"w1": 10, "w2": 3, "w3": 5},
#     "combination_4": {"w1": 5, "w2": 10, "w3": 3},
#     "combination_5": {"w1": 3, "w2": 10, "w3": 5},
#     "combination_6": {"w1": 5, "w2": 3, "w3": 10},
#     "combination_7": {"w1": 3, "w2": 5, "w3": 10},
#     "combination_8": {"w1": 100, "w2": 5, "w3": 3},
#     "combination_9": {"w1": 5, "w2": 100, "w3": 3},
#     "combination_10": {"w1": 5, "w2": 3, "w3": 100},
#     "combination_11": {"w1": 1, "w2": 1, "w3": 1},
# }

# # 重み設定　（カスタム目的関数その1）
# weight_sets = {
#     "combination_1": {"w1": 10, "w2": 45, "w3": 45, "w4": 50},
#     # "combination_2": {"w1": 10, "w2": 5, "w3": 3, "w4": },
#     # "combination_3": {"w1": 10, "w2": 3, "w3": 5, "w4": },
#     # "combination_4": {"w1": 5, "w2": 10, "w3": 3, "w4": },
#     # "combination_5": {"w1": 3, "w2": 10, "w3": 5, "w4": },
#     # "combination_6": {"w1": 5, "w2": 3, "w3": 10, "w4": },
#     # "combination_7": {"w1": 3, "w2": 5, "w3": 10, "w4": },
#     # "combination_8": {"w1": 100, "w2": 5, "w3": 3, "w4": 10},
#     "combination_9": {"w1": 5, "w2": 100, "w3": 3, "w4": 12},
#     # "combination_10": {"w1": 5, "w2": 3, "w3": 100, "w4": },
#     "combination_11": {"w1": 1, "w2": 1, "w3": 1, "w4": 1},
# }

# 重み設定　（カスタム目的関数その1）
weight_sets = {
    # === combination_1 ===
    "combination_1_w4_w1": {"w1": 10, "w2": 45, "w3": 45, "w4": 10},
    "combination_1_w4_w2": {"w1": 10, "w2": 45, "w3": 45, "w4": 45},
    "combination_1_w4_w3": {"w1": 10, "w2": 45, "w3": 45, "w4": 45},
    "combination_1_w4_mean": {"w1": 10, "w2": 45, "w3": 45, "w4": 33},

    # === combination_2 ===
    "combination_2_w4_w1": {"w1": 10, "w2": 5, "w3": 3, "w4": 10},
    "combination_2_w4_w2": {"w1": 10, "w2": 5, "w3": 3, "w4": 5},
    "combination_2_w4_w3": {"w1": 10, "w2": 5, "w3": 3, "w4": 3},
    "combination_2_w4_mean": {"w1": 10, "w2": 5, "w3": 3, "w4": 6},

    # === combination_3 ===
    "combination_3_w4_w1": {"w1": 10, "w2": 3, "w3": 5, "w4": 10},
    "combination_3_w4_w2": {"w1": 10, "w2": 3, "w3": 5, "w4": 3},
    "combination_3_w4_w3": {"w1": 10, "w2": 3, "w3": 5, "w4": 5},
    "combination_3_w4_mean": {"w1": 10, "w2": 3, "w3": 5, "w4": 6},

    # === combination_4 ===
    "combination_4_w4_w1": {"w1": 5, "w2": 10, "w3": 3, "w4": 5},
    "combination_4_w4_w2": {"w1": 5, "w2": 10, "w3": 3, "w4": 10},
    "combination_4_w4_w3": {"w1": 5, "w2": 10, "w3": 3, "w4": 3},
    "combination_4_w4_mean": {"w1": 5, "w2": 10, "w3": 3, "w4": 6},

    # === combination_5 ===
    "combination_5_w4_w1": {"w1": 3, "w2": 10, "w3": 5, "w4": 3},
    "combination_5_w4_w2": {"w1": 3, "w2": 10, "w3": 5, "w4": 10},
    "combination_5_w4_w3": {"w1": 3, "w2": 10, "w3": 5, "w4": 5},
    "combination_5_w4_mean": {"w1": 3, "w2": 10, "w3": 5, "w4": 6},

    # === combination_6 ===
    "combination_6_w4_w1": {"w1": 5, "w2": 3, "w3": 10, "w4": 5},
    "combination_6_w4_w2": {"w1": 5, "w2": 3, "w3": 10, "w4": 3},
    "combination_6_w4_w3": {"w1": 5, "w2": 3, "w3": 10, "w4": 10},
    "combination_6_w4_mean": {"w1": 5, "w2": 3, "w3": 10, "w4": 6},

    # === combination_7 ===
    "combination_7_w4_w1": {"w1": 3, "w2": 5, "w3": 10, "w4": 3},
    "combination_7_w4_w2": {"w1": 3, "w2": 5, "w3": 10, "w4": 5},
    "combination_7_w4_w3": {"w1": 3, "w2": 5, "w3": 10, "w4": 10},
    "combination_7_w4_mean": {"w1": 3, "w2": 5, "w3": 10, "w4": 6},

    # === combination_8 ===
    "combination_8_w4_w1": {"w1": 100, "w2": 5, "w3": 3, "w4": 100},
    "combination_8_w4_w2": {"w1": 100, "w2": 5, "w3": 3, "w4": 5},
    "combination_8_w4_w3": {"w1": 100, "w2": 5, "w3": 3, "w4": 3},
    "combination_8_w4_mean": {"w1": 100, "w2": 5, "w3": 3, "w4": 36},

    # === combination_9 ===
    "combination_9_w4_w1": {"w1": 5, "w2": 100, "w3": 3, "w4": 5},
    "combination_9_w4_w2": {"w1": 5, "w2": 100, "w3": 3, "w4": 100},
    "combination_9_w4_w3": {"w1": 5, "w2": 100, "w3": 3, "w4": 3},
    "combination_9_w4_mean": {"w1": 5, "w2": 100, "w3": 3, "w4": 36},
    "combination_9_w4_additional": {"w1": 5, "w2": 100, "w3": 3, "w4": 10},

    # === combination_10 ===
    "combination_10_w4_w1": {"w1": 5, "w2": 3, "w3": 100, "w4": 5},
    "combination_10_w4_w2": {"w1": 5, "w2": 3, "w3": 100, "w4": 3},
    "combination_10_w4_w3": {"w1": 5, "w2": 3, "w3": 100, "w4": 100},
    "combination_10_w4_mean": {"w1": 5, "w2": 3, "w3": 100, "w4": 36},

    # === combination_11 ===
    "combination_11_w4_w1": {"w1": 1, "w2": 1, "w3": 1, "w4": 1},
    "combination_11_w4_w2": {"w1": 1, "w2": 1, "w3": 1, "w4": 1},
    "combination_11_w4_w3": {"w1": 1, "w2": 1, "w3": 1, "w4": 1},
    "combination_11_w4_mean": {"w1": 1, "w2": 1, "w3": 1, "w4": 1},

    # === 支配ケース（4変数すべて） ===
    "dominant_w1": {"w1": 100, "w2": 5, "w3": 3, "w4": 5},
    "dominant_w2": {"w1": 5, "w2": 100, "w3": 3, "w4": 5},
    "dominant_w3": {"w1": 5, "w2": 3, "w3": 100, "w4": 5},
    "dominant_w4": {"w1": 5, "w2": 3, "w3": 5, "w4": 100},
}


# cfg = weight_sets["combination_1"]
# cfg = weight_sets["combination_7"]

# スタート地点・ゴール地点の設定
pairs = [(1, 19), (4, 140), (15, 452)] # 実験1 すべて偶数長でつなげるパターン
# pairs = [(1, 29), (4, 140), (15, 452)] # 実験2 一つだけ奇数、あとは偶数のパターン（偶奇が異なるパターン）
# pairs = [] # 実験3 ランダムに点を生成


# 参考のために、それぞれのスタート・ゴールの最短経路長を出しておく。
shortest_dist = []
for p in pairs:
    d, prevs = dijkstra(p[0], grid_size**3, graph_3d.to)
    path = get_path(p[0], p[1], prevs)
    path = path[::-1]
    print('len(path) =', len(path))
    print('path =', path)
    


param_names = []
sds = []
path1_lengths = []
path2_lengths = []
path3_lengths = []
n_crosses = []
n_corners = []

w1s = []
w2s = []
w3s = []
w4s = []

all_pathes = {}

for key in weight_sets.keys():
    weights = weight_sets[key]
    param_name, sd, path1_length, path2_length, path3_length, n_cross, paths, n_corner = main_3d_inspection(pairs, key, **weights, do_visualize=False)
    all_pathes[param_name] = paths
    param_names.append(param_name)
    sds.append(sd)
    path1_lengths.append(path1_length)
    path2_lengths.append(path2_length)
    path3_lengths.append(path3_length)
    n_crosses.append(n_cross)
    n_corners.append(n_corner)

    # 重み
    w1s.append(weights["w1"])
    w2s.append(weights["w2"])
    w3s.append(weights["w3"])
    if "w4" in weights.keys():
        w4s.append(weights["w4"])


In [ ]:
import pandas as pd

# columns = ["seed", "param_name", "w1", "w2", "w3", "w4", "path1_length", "path2_length", "path3_length", "n_cross"]
columns = ["seed", "param_name", "w1", "w2", "w3", "path1_length", "path2_length", "path3_length", "n_cross"]

# 空の DataFrame を作成
df = pd.DataFrame(columns=columns)

df["param_name"] = param_names
df["w1"] = w1s
df["w2"] = w2s
df["w3"] = w3s
if "w4" in columns:
    df["w4"] = w4s
df["seed"] = sds
df["path1_length"] = path1_lengths
df["path2_length"] = path2_lengths
df["path3_length"] = path3_lengths
df["n_cross"] = n_crosses
df["n_corner"] = n_corners 


In [ ]:
display(df)

In [ ]:
# n_crossが0のものだけ取る
df[df['n_cross']==0]

In [ ]:
# 曲げ回数の平均値
df['n_corner'].mean()

In [ ]:
assert False

---
### 可視化

In [ ]:
# 特定の実験で得られたpathsを表示する。（実験のパラメータ名称を指定する）
# param_name = "combination_9_w4_additional"
# param_name = "combination_9_w4_w3"
param_name = "dominant_w2"
# param_name = "combination_1"

paths = all_pathes[param_name]

# 結果の表示(1本ずつの表示)
for i, path in enumerate(paths):
    print(f"w1: {weight_sets[param_name]["w1"]}, w2: {weight_sets[param_name]["w2"]}, w3: {weight_sets[param_name]["w3"]}")
    print('len(path) =', len(path))
    print('n_corner =', counting_corner(grid_size, path))
    if i == 0:
        path1_length = len(path)
    elif i == 1:
        path2_length = len(path)
    elif i == 2:
        path3_length = len(path)

    vis_gridpath_3d(grid_size, [path], obs_nodes=graph_3d.obs, saveflg=False)

# 結果の表示(3本まとめて表示)
vis_gridpath_3d(grid_size, paths, obs_nodes=graph_3d.obs, saveflg=False)

In [ ]:
vis_gridpath_3d(10, paths=None, obs_nodes=graph_3d.obs)